- (1) 섹션 & 링크
- (2) topic / url / section
- (3) sub 페이지 (기사 내용)
언론사명, 제목, 날짜/시간, 기사 내용, 작성자


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

from urllib.request import urlopen
import pandas as pd
import bs4
import requests

In [2]:
url =  'https://news.daum.net'
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html, 'html.parser')
# bs_obj

### (1) 다음 뉴스 섹션 , url 추출

In [3]:
ul = bs_obj.find('ul', {'class':'gnb_comm'})
# ul

In [4]:
lis = ul.select('li')

section = []
link = []

for li in lis:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append('https://news.daum.net'+a_tag['href'])

section_menu_df = pd.DataFrame({'section':section, 'link':link})
section_menu_df

,section,link
0,홈,https://news.daum.net/
1,사회,https://news.daum.net/society
2,정치,https://news.daum.net/politics
3,경제,https://news.daum.net/economic
4,국제,https://news.daum.net/foreign
5,문화,https://news.daum.net/culture
6,IT,https://news.daum.net/digital
7,연재,https://news.daum.net/series/
8,포토,https://news.daum.net/photo
9,팩트체크,https://news.daum.nethttps://focus.daum.net/ch...


### (2) 섹션별 헤드라인 제목, Url추출

In [5]:
# class tit_g , atag, text

In [6]:
def get_topic(url, section):
    # 접속
    headers = {'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'}
    # 빈 리스트 생성
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')

    # topic list
    topic_list = bs_obj.findAll('strong', {'class':'tit_g'})

    topic_title = []
    topic_link = []

    for topic in topic_list:
        try:
            a = topic.find('a')
            topic_title.append(a.text)
            topic_link.append(a['href'])
        except:
            # print('topic 추출 중 오류발생')
            pass

    return ({'topic':topic_title, 'url':topic_link, 'section':section})

In [7]:
# 모든 섹션 헤드라인 데이터 프레임 생성
all_topic_df = pd.DataFrame({
    'topic':[],
    'url':[],
    'section':[]
})

for i in range(1,7):
    df = pd.DataFrame(get_topic(section_menu_df['link'][i], section_menu_df['section'][i]))
    all_topic_df = pd.concat([all_topic_df, df], axis=0, ignore_index=True)

In [8]:
all_topic_df

,topic,url,section
0,"박근혜, 김기현 만나 ""총선 승리""...문재인, 이재명에게 ""단식 멈춰야""",https://v.daum.net/v/20230914091927496,사회
1,"서울시, 월 6만5천 원에 '무제한 교통카드'...과제는?",https://v.daum.net/v/20230914091325295,사회
2,구직 포기하는 청년들?...취업자 수 '뚝',https://v.daum.net/v/20230914090145876,사회
3,김종대 “북러회담은 격변…북중러 ‘빅딜’ 향해 갈 것”,https://v.daum.net/v/20230914085348572,사회
4,"""자전거가 우선"" 지하철서 살해 협박까지?",https://v.daum.net/v/20230914085215534,사회
5,모로코 '지진' 리비아 '홍수'…대한적십자사 구호 활동,https://v.daum.net/v/20230914083207991,사회
6,"中외교수장, 러시아로…김정은 이어 시진핑 주목",https://v.daum.net/v/20230914082401821,사회
7,손 글씨로 잇는 너와 나의 다리,https://v.daum.net/v/20230914082224787,사회
8,"""이종섭 사퇴? 장관들, 런닝맨 찍나"" vs ""개각했으니 이재명 단식 풀어야""",https://v.daum.net/v/20230914090801111,정치
9,"정대철 ""윤석열 대통령, 제일 잘 한게 외교""",https://v.daum.net/v/20230914080113355,정치


In [11]:
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'}
    # 빈 리스트 생성
    paper_list, title_list, datetime_list, writing_list, writer_list = [ [] for _ in range(5) ]

    try:
        # 접속
        res = requests.get(url, headers=headers)
        html = res.text
        bs_obj = bs4.BeautifulSoup(html, 'html.parser')

        # 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자 리스트 생성
        paper_list.append(bs_obj.select_one('#kakaoServiceLogo').text)
        title_list.append(bs_obj.find('h3', {'class':'tit_view'}).text)
        datetime_list.append(bs_obj.find('span', {'class':'num_date'}).text)
        writing_list.append(bs_obj.find('div', {'class':'article_view'}).select('p').text.replace('\n',''))
        writer_list.append(bs_obj.find('span',{'class':'txt_info'}).text)
        # writer = bs_obj.find('p', attr={'dmcf-pid':'b2Lt981ihL'}).text
        # if writer != None:
        #     writer_list.append(writer)
        # else:
        #     writer_list.append('없음')
        # writer_list.append(bs_obj.find('p',attr={'dmcf-pid':'b2Lt981ihL'}).text)
        
        
        ### test
        # test = bs_obj.find('div', {'class' : 'article_view'}).select("section p[dmcf-ptype='general']")[-1].text          
        # # print("===" + test)
        # word_list = ['기자', '특파원', '논설위원']
        # # for word in word_list:
        # #     if word in test:
        # #         print("===" + test)
    
        # if any(word in test for word in word_list):
        #     print("===" + test[-31:])
        # else:
        #     print("미상")

    
    except:
        # print('sub_news 에러 발생!')
        # writer_list.append('')
        pass
        
    # dictionary로 만들어서 반환
    sub_news_dict = {
        'paper' : paper_list,
        'title' : title_list,
        'datetime' : datetime_list,
        'writing' : writing_list,
        'writer' : writer_list
    }
    return sub_news_dict

In [12]:
all_sub_news_info_df = pd.DataFrame({
    'paper' : [],
    'title' : [],
    'datetime' : [],
    'writing' : [],
    'writer' : []
})

for url in all_topic_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)
all_sub_news_info_df

ValueError: All arrays must be of the same length

In [ ]:
all_sub_news_info_df.to_csv('./crawl_data/daum_sub_news_section.csv', index=0)

In [ ]:
all_sub_news_info_df = pd.read_csv('./crawl_data/daum_sub_news_section.csv')
all_sub_news_info_df